In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

In [2]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [7]:
train=pd.read_csv("/content/fashion-mnist_train.csv")
test=pd.read_csv("/content/fashion-mnist_test.csv")

In [8]:
y_train=train.iloc[:,0].values
X_train=train.iloc[:,1:].values
y_test=test.iloc[:,0].values
X_test=test.iloc[:,1:].values

In [9]:
X_train=X_train/255.0
X_test=X_test/255.0

In [10]:
train_df=CustomDataset(X_train,y_train)

In [11]:
train_df[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0157, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2431, 0.2392, 0.0824, 0.1137, 0.0902,
         0.2000, 0.5333, 0.2392, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [12]:
test_df=CustomDataset(X_test,y_test)
test_df[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0353, 0.0314,
         0.0000, 0.0000, 0.1333, 0.1137, 0.0275, 0.0000, 0.0431, 0.0941, 0.0000,
         0.0000, 0.0118, 0.0118, 0.0039, 0.0000, 0.0039, 0.0039, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0157, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1725, 0.3451, 0.3882, 0.4784, 0.4824, 0.3137,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0039, 0.0039, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0078, 0.0000, 0.0000, 0.0000,
         0.0118, 0.1804, 0.6824, 0.9765, 0.2627, 0.0000, 0.3686, 0.8235, 0.2392,
         0.0549, 0.8314, 0.6157, 0.1451, 0.0000, 0.0000, 0.0000, 0.0000, 0.0039,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0078, 0.0078, 0.0000, 0.0902,
         0.6588, 0.8078, 0.9490, 0.9373, 0.9333, 0.8392, 0.4902, 0.2392, 0.4431,
         0.2902, 0.5216, 0.9255, 0.9333, 0.9255, 0.7961, 0.7216, 0.0784, 0.0000,
         0.0039, 0.0000, 0.0

In [13]:
train_loader=DataLoader(train_df,batch_size=64,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_df,batch_size=64,shuffle=False,pin_memory=True)

In [14]:
class CustomNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()

    self.model=nn.Sequential(
        nn.Linear(num_features,256),
        nn.ReLU(),
        nn.Linear(256,64),
        nn.ReLU(),
        nn.Linear(64,10)
        # Need not add softmax explicitely, it is automatically handled
    )

  def forward(self,X):
    return self.model(X)

In [15]:
epochs=50
lr=0.1

In [16]:
X_train.shape

(60000, 784)

In [17]:
model=CustomNN(X_train.shape[1])
model=model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=lr)

In [15]:
for epoch in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    outputs=model(batch_features)
    loss=criterion(outputs,batch_labels)
    loss.backward()
    optimizer.step() # updates model params
    optimizer.zero_grad() # reset gradients to 0
    total_loss=total_loss+loss.item()
  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(train_loader)}")

Epoch: 1, Loss: 0.6911799310843574
Epoch: 2, Loss: 0.4441897935990586
Epoch: 3, Loss: 0.3917527844720304
Epoch: 4, Loss: 0.3647293443602921
Epoch: 5, Loss: 0.34045874154262706
Epoch: 6, Loss: 0.32425739117332103
Epoch: 7, Loss: 0.31109482096806007
Epoch: 8, Loss: 0.29794868803037
Epoch: 9, Loss: 0.28585333578082034
Epoch: 10, Loss: 0.2753752179261146
Epoch: 11, Loss: 0.2669626146411972
Epoch: 12, Loss: 0.2589573455628937
Epoch: 13, Loss: 0.25401772401416733
Epoch: 14, Loss: 0.24484917496456138
Epoch: 15, Loss: 0.23713299310378938
Epoch: 16, Loss: 0.23215674214176277
Epoch: 17, Loss: 0.22412743175954325
Epoch: 18, Loss: 0.22010419183750268
Epoch: 19, Loss: 0.21279550645190642
Epoch: 20, Loss: 0.20864780854060452
Epoch: 21, Loss: 0.20457955574525444
Epoch: 22, Loss: 0.19919978669171395
Epoch: 23, Loss: 0.19627999318148026
Epoch: 24, Loss: 0.1910352736219033
Epoch: 25, Loss: 0.1862822117955128
Epoch: 26, Loss: 0.1805701792907359
Epoch: 27, Loss: 0.176399765845968
Epoch: 28, Loss: 0.172512

In [16]:
model.eval()

CustomNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [17]:
total=0
correct=0
with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    batch_features, batch_labels=batch_features.to(device), batch_labels.to(device)
    outputs=model(batch_features)
    _, predicted=torch.max(outputs, 1)
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(correct/total)

0.8984
